<h1> Public and School Holidays in Germany's Federal States </h1>

Public holidays are — with the exception of one federal holiday — determined individually by each of the federal states. The same applies to school holidays. Summer school holidays are even deliberately staggered to start and end on different dates in order to reduce the strain on the country’s highway system caused by vacation traffic.


In [6]:
from pathlib import Path
import sys, importlib
import pandas as pd

# Make repo importable and load config
PROJECT_ROOT = Path.cwd().parent
if str(PROJECT_ROOT) not in sys.path:
    sys.path.insert(0, str(PROJECT_ROOT))

import src.config as cfg
cfg = importlib.reload(cfg)
from src.de_hol_df import make_de_hol_df

from src.de_hol_df import (make_de_hol_df, update_public_DE_hol_df_fct, update_school_DE_hol_df_fct, )
from src.data_modules.german_school_semesters import SEMESTER_REGISTRY

<h2> DataFrame Initialization</h2>

Here, we set up the `DE_hol_df` DataFrame, starting from `2015-01-01`.  It contains the following datetime columns:


`local_date: datetime64[ns]`: A naive daily date, no timezone, just a day label  
`local_start: datetime64[ns, Europe/Berlin]`: The **start** of the corresponding day  in  tz-aware local time   
`local_end: datetime64[ns, Europe/Berlin] `: The **end** of the corresponding day  in  tz-aware local time  
`utc_start: datetime64[ns, UTC] ` : The **start** of the corresponding day  in  tz-aware UTC  
`utc_end: datetime64[ns, UTC] `:  The **end** of the corresponding day  in  tz-aware UTC  
  
  In addition, each state gets two columns indicating whether a given day is a public holiday or a school-free day using  `UInt8`flags.

In [4]:
from src.de_hol_df import make_de_hol_df

DE_hol_df = make_de_hol_df(
    start="2015-01-01",
    end="2035-12-31",
    tz="Europe/Berlin",
)

In [5]:
DE_hol_df

,local_date,local_start,local_end,utc_start,utc_end,DE_BW_hol,DE_BW_school_free,DE_BY_hol,DE_BY_school_free,DE_BE_hol,...,DE_SL_hol,DE_SL_school_free,DE_SN_hol,DE_SN_school_free,DE_ST_hol,DE_ST_school_free,DE_SH_hol,DE_SH_school_free,DE_TH_hol,DE_TH_school_free
0,2015-01-01,2015-01-01 00:00:00+01:00,2015-01-02 00:00:00+01:00,2014-12-31 23:00:00+00:00,2015-01-01 23:00:00+00:00,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,2015-01-02,2015-01-02 00:00:00+01:00,2015-01-03 00:00:00+01:00,2015-01-01 23:00:00+00:00,2015-01-02 23:00:00+00:00,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,2015-01-03,2015-01-03 00:00:00+01:00,2015-01-04 00:00:00+01:00,2015-01-02 23:00:00+00:00,2015-01-03 23:00:00+00:00,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,2015-01-04,2015-01-04 00:00:00+01:00,2015-01-05 00:00:00+01:00,2015-01-03 23:00:00+00:00,2015-01-04 23:00:00+00:00,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,2015-01-05,2015-01-05 00:00:00+01:00,2015-01-06 00:00:00+01:00,2015-01-04 23:00:00+00:00,2015-01-05 23:00:00+00:00,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7665,2035-12-27,2035-12-27 00:00:00+01:00,2035-12-28 00:00:00+01:00,2035-12-26 23:00:00+00:00,2035-12-27 23:00:00+00:00,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7666,2035-12-28,2035-12-28 00:00:00+01:00,2035-12-29 00:00:00+01:00,2035-12-27 23:00:00+00:00,2035-12-28 23:00:00+00:00,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7667,2035-12-29,2035-12-29 00:00:00+01:00,2035-12-30 00:00:00+01:00,2035-12-28 23:00:00+00:00,2035-12-29 23:00:00+00:00,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7668,2035-12-30,2035-12-30 00:00:00+01:00,2035-12-31 00:00:00+01:00,2035-12-29 23:00:00+00:00,2035-12-30 23:00:00+00:00,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


<h2>Update the DataFrame's Public Holiday Columns </h2>

This section populates the **public holiday flags** in `DE_hol_df` using the [`python-holidays`](https://pypi.org/project/holidays/) library:   
For each German federal state (`DE_BW`, `DE_BY`, …), the function `update_public_DE_hol_df_fct` marks all official public holidays as `1` in the corresponding `<STATE>_hol` column.   
By default, these days are also flagged as **school-free** — the function automatically copies each public holiday into the matching `<STATE>_school_free` column.  


In [ ]:
!{sys.executable} -m pip install holidays


In [11]:
from pathlib import Path
import sys
sys.path.insert(0, str(Path.cwd().parent))
import pandas as pd

# Populate public holiday flags using python-holidays
# (holiday_implies_school_free=True means holidays get flagged as school-free)
DE_hol_df = update_public_DE_hol_df_fct(DE_hol_df, holiday_implies_school_free=True)


In [ ]:
DE_hol_df

<h2> The School Holidays of Germany's 16 Federal States </h2>


<h3>Examples: Data for 2015 and 2016 </h3>

Not all school holidays are determined by the state governments. There are also so-called “Bewegliche Ferientage” (“movable school holidays”) that can be scheduled individually by each school. Their placement often follows local customs, which means that certain days may effectively function as de facto semi school holidays. The Friday after Ascension Day is a typical example.

<h4>🎓 The year 2015 </h4>

The calendar year 2015 comprises two semesters of two different school-years. 


| state | year | Last year's Christmas | Winter | Easter | Pentecost       | Summer | Fall | Christmas          |
|:-----:|:----:|:---------------------:|:------:|:------:|:---------------:|:------:|:----:|:------------------:|
| (Land)|      | (first days in Jan)   |        |        | (+ Post-Ascension Fri.) ||      | (last days in Dec) |
| `BW` | 2015 | `2015-01-01` --- `2015-01-05` | – | `2015-03-30` --- `2015-04-10` | `2015-05-26` --- `2015-06-06` | `2015-07-30` --- `2015-09-12` | `2015-11-02` --- `2015-11-06` | `2015-12-23` --- `2015-12-31` |
| `BY` | 2015 | `2015-01-01` --- `2015-01-05` | `2015-02-16` --- `2015-02-20` | `2015-03-30` --- `2015-04-11` | `2015-05-26` --- `2015-06-05` | `2015-07-31`―`2015-09-14` | `2015-11-02`―`2015-11-07` | `2015-12-24`―`2015-12-31` |
| `BE` | 2015 | `2015-01-01` --- `2015-01-02` | `2015-02-02` --- `2015-02-07` | `2015-03-30` --- `2015-04-11` | `2015-05-15`, `2015-05-26` | `2015-07-16` --- `2015-08-28` | `2015-10-19` --- `2015-10-31` | `2015-12-23` --- `2015-12-31` |
| `BB` | 2015 | `2015-01-01` --- `2015-01-02` | `2015-02-02` --- `2015-02-07` | `2015-04-01` --- `2015-04-11` | `2015-05-15` | `2015-07-16`―`2015-08-28` | `2015-10-19` --- `2015-10-30` | `2015-12-23` --- `2015-12-31` |
| `HB` | 2015 | `2015-01-01` --- `2015-01-05` | `2015-02-02` --- `2015-02-03` | `2015-03-25`―`2015-04-10` | `2015-05-15`, `2015-05-26` | `2015-07-23` --- `2015-09-02` | `2015-10-19` --- `2015-10-31` | `2015-12-23` --- `2015-12-31` |
| `HH` | 2015 | `2015-01-01` --- `2015-01-06` | `2015-01-30` | `2015-03-02`―`2015-03-13` | `2015-05-11`―`2015-05-15` | `2015-07-16`―`2015-08-26` | `2015-10-19`―`2015-10-30` | `2015-12-23`―`2015-12-31` |
| `HE` | 2015 | `2015-01-01` --- `2015-01-10` | – | `2015-03-30`―`2015-04-11` | – | `2015-07-27`―`2015-09-04` | `2015-10-19` --- `2015-10-31` | `2015-12-23`―`2015-12-31` |
| `MV` | 2015 | `2015-01-01` --- `2015-01-02` | `2015-02-02`―`2015-02-14` | `2015-03-30`―`2015-04-08` | `2015-05-22` --- `2015-05-26` | `2015-07-20`―`2015-08-29` | `2015-10-24`―`2015-10-30` | `2015-12-23`―`2015-12-31` |
| state | year | Last year's Christmas | Winter | Easter | Pentecost       | Summer            | Fall | Christmas |
| `NI` | 2015 | `2015-01-01` --- `2015-01-05` | `2015-02-02` --- `2015-02-03` | `2015-03-25` --- `2015-04-10` | `2015-05-15`, `2015-05-26` | `2015-07-23`―`2015-09-02` | `2015-10-19`―`2015-10-31` | `2015-12-23` --- `2015-12-31` |
| `NW` | 2015 | `2015-01-01` --- `2015-01-06` | – | `2015-03-30` --- `2015-04-11` | `2015-05-26` | `2015-06-29` --- `2015-08-11` | `2015-10-05`―`2015-10-17` | `2015-12-23`―`2015-12-31` |
| `RP` | 2015 | `2015-01-01` --- `2015-01-07` | – | `2015-03-26`―`2015-04-10` | `2015-05-15`, `2015-06-05` | `2015-07-27` --- `2015-09-04` | `2015-10-19`―`2015-10-30` | `2015-12-23`―`2015-12-31` |
| `SL` | 2015 | `2015-01-01` --- `2015-01-07` | `2015-02-16`―`2015-02-21` | `2015-03-30` --- `2015-04-11` | – | `2015-07-27` --- `2015-09-05` | `2015-10-19`―`2015-10-31` | `2015-12-23`―`2015-12-31` |
| `SN` | 2015 | `2015-01-01` --- `2015-01-03` | `2015-02-09` --- `2015-02-21` | `2015-04-02` --- `2015-04-11` | `2015-05-15` | `2015-07-13`―`2015-08-21` | `2015-10-12` --- `2015-10-24` | `2015-12-23` --- `2015-12-31` |
| `ST` | 2015 | `2015-01-01` --- `2015-01-05` | `2015-02-02` --- `2015-02-14` | `2015-04-02` | `2015-05-15`―`2015-05-23` | `2015-07-13`―`2015-08-26` | `2015-10-17`―`2015-10-24` | `2015-12-23` --- `2015-12-31` |
| `SH` | 2015 | `2015-01-01`―`2015-01-06` | – | `2015-04-01` --- `2015-04-17` | `2015-05-15` | `2015-07-20` --- `2015-08-29` | `2015-10-19`―`2015-10-31` | `2015-12-23`―`2015-12-31` |
| `TH` | 2015 | `2015-01-01`―`2015-01-03` | `2015-02-02`―`2015-02-07` | `2015-03-30`―`2015-04-11` | `2015-05-15` | `2015-07-13`―`2015-08-21` | `2015-10-05` --- `2015-10-17` | `2015-12-23` --- `2015-12-31` |

<h4>🎓 The year 2016 </h4>

| state | year | Last year's Christmas | Winter | Easter | Pentecost       | Summer            | Fall                   | Christmas          |
|:-----:|:----:|:----------------------:|:------:|:------:|:----------------:|:------------------:|:----------------------:|:------------------:|
| (Land) |  | (first days in Jan) |         |      | (+ Post-Ascension Fri.) |   |   | (last days in Dec) |
| `BW` | 2016 | `2016-01-01` ― `2016-01-09` | – | `2016-03-29` ― `2016-04-02` | `2016-05-17` ― `2016-05-28` | `2016-07-28` ― `2016-09-10` | `2016-11-02` ― `2016-11-04` | `2016-12-23` ― `2016-12-31` |
| `BY` | 2016 | `2016-01-01` ― `2016-01-05` | `2016-02-08` ― `2016-02-12` | `2016-03-21` ― `2016-04-01` | `2016-05-17` ― `2016-05-28` | `2016-07-30` ― `2016-09-12` | `2016-10-31` ― `2016-11-04` | `2016-12-24` ― `2016-12-31` |
| `BE` | 2016 | `2016-01-01` ― `2016-01-02` | `2016-02-01` ― `2016-02-06` | `2016-03-21` ― `2016-04-02` | `2016-05-06`, `2016-05-17` | `2016-07-20` ― `2016-09-02` | `2016-10-17` ― `2016-10-28` | `2016-12-23` ― `2016-12-31` |
| `BB` | 2016 | `2016-01-01` ― `2016-01-02` | `2016-02-01` ― `2016-02-06` | `2016-03-23` ― `2016-04-02` | `2016-05-06`, `2016-05-17` | `2016-07-21` ― `2016-09-03` | `2016-10-17` ― `2016-10-30` | `2016-12-23` ― `2016-12-31` |
| `HB` | 2016 | `2016-01-01` ― `2016-01-06` | `2016-01-28` ― `2016-01-29` | `2016-03-18` ― `2016-04-02` | `2016-05-06`, `2016-05-17` | `2016-06-23` ― `2016-08-03` | `2016-10-04` ― `2016-10-15` | `2016-12-23` ― `2016-12-31` |
| `HH` | 2016 | `2016-01-01` ― `2016-01-01` | `2016-01-29` | `2016-03-07` ― `2016-03-18` | `2016-05-06`, `2016-05-17` ― `2016-05-20` | `2016-07-21` ― `2016-08-31` | `2016-10-17` ― `2016-10-28` | `2016-12-23` ― `2016-12-31` |
| `HE` | 2016 | `2016-01-01` ― `2016-01-09` | – | `2016-03-29` ― `2016-04-09` | – | `2016-07-18` ― `2016-08-26` | `2016-10-17` ― `2016-10-29` | `2016-12-22` ― `2016-12-31` |
| `MV` | 2016 | `2016-01-01` ― `2016-01-02` | `2016-02-01` ― `2016-02-13` | `2016-03-21` ― `2016-03-30` | `2016-05-14` ― `2016-05-17` | `2016-07-25` ― `2016-09-03` | `2016-10-17` ― `2016-10-28` | `2016-12-22` ― `2016-12-31` |
| `NI` | 2016 | `2016-01-01` ― `2016-01-06` | `2016-01-28` ― `2016-01-29` | `2016-03-18` ― `2016-04-02` | `2016-05-06`, `2016-05-17` | `2016-06-23` ― `2016-08-03` | `2016-10-17` ― `2016-10-28` | `2016-12-23` ― `2016-12-31` |
| `NW` | 2016 | `2016-01-01` ― `2016-01-06` | – | `2016-03-21` ― `2016-04-02` | `2016-05-17` | `2016-07-11` ― `2016-08-23` | `2016-10-10` ― `2016-10-21` | `2016-12-23` ― `2016-12-31` |
| `RP` | 2016 | `2016-01-01` ― `2016-01-08` | – | `2016-03-18` ― `2016-04-01` | `2016-05-06`, `2016-05-27` | `2016-07-18` ― `2016-08-26` | `2016-10-10` ― `2016-10-21` | `2016-12-23` ― `2016-12-31` |
| `SL` | 2016 | `2016-01-01` ― `2016-01-02` | `2016-02-08` ― `2016-02-13` | `2016-03-29` ― `2016-04-09` | – | `2016-07-18` ― `2016-08-27` | `2016-10-10` ― `2016-10-22` | `2016-12-21` ― `2016-12-31` |
| `SN` | 2016 | `2016-01-01` ― `2016-01-02` | `2016-02-08` ― `2016-02-20` | `2016-03-25` ― `2016-04-02` | `2016-05-06` | `2016-06-27` ― `2016-08-05` | `2016-10-03` ― `2016-10-15` | `2016-12-23` ― `2016-12-31` |
| `ST` | 2016 | `2016-01-01` ― `2016-01-05` | `2016-02-01` ― `2016-02-10` | `2016-03-24` | `2016-05-06` ― `2016-05-14` | `2016-06-27` ― `2016-08-10` | `2016-10-04` ― `2016-10-15` | `2016-12-19` ― `2016-12-31` |
| `SH` | 2016 | `2016-01-01` ― `2016-01-06` | – | `2016-03-24` ― `2016-04-09` | `2016-05-06` | `2016-07-25` ― `2016-09-03` | `2016-10-17` ― `2016-10-29` | `2016-12-23` ― `2016-12-31` |
| `TH` | 2016 | `2016-01-01` ― `2016-01-02` | `2016-02-01` ― `2016-02-06` | `2016-03-24` ― `2016-04-02` | `2016-05-06` | `2016-06-27` ― `2016-08-10` | `2016-10-10` ― `2016-10-22` | `2016-12-23` ― `2016-12-31` |


<h3> School Holiday Dictionaries  </h3>

We organize the data as follows:   
The time is divided into **school semesters**, as defined individually by each state.
Accordingly, `DE_hol_2015_1_dict` contains, for each state, its school holidays for the **first semester of 2015**, which typically corresponds to the **second semester of the 2014/2015 school year**.
This means that a given day may belong to different semesters depending on the state.
Each state abbreviation serves as a **key**, and its **value** is a list of **intervals**.
Single-day holidays are represented as one-day intervals.   
   
All semester dictionaries covering the period from **2015-01-01** to **2020-12-31** have been consolidated in  
`src/data_modules/german_school_semesters.py`  
so that they can be easily accessed by the update function.

In [ ]:
DE_hol_2015_1_dict = {
    'BW': [('2015-01-01','2015-01-05'), ('2015-03-30','2015-04-10'), ('2015-05-26','2015-06-06'), ('2015-07-30','2015-09-12')],
    'BY': [('2015-01-01','2015-01-05'), ('2015-02-16','2015-02-20'), ('2015-03-30','2015-04-11'), ('2015-05-26','2015-06-05'), ('2015-07-31','2015-09-14')],
    'BE': [('2015-01-01','2015-01-02'), ('2015-02-02','2015-02-07'), ('2015-03-30','2015-04-11'), ('2015-05-15','2015-05-15'), ('2015-05-26','2015-05-26'), ('2015-07-16','2015-08-28')],
    'BB': [('2015-01-01','2015-01-02'), ('2015-02-02','2015-02-07'), ('2015-04-01','2015-04-11'), ('2015-05-15','2015-05-15'), ('2015-07-16','2015-08-28')],
    'HB': [('2015-01-01','2015-01-05'), ('2015-02-02','2015-02-03'), ('2015-03-25','2015-04-10'), ('2015-05-15','2015-05-15'), ('2015-05-26','2015-05-26'), ('2015-07-23','2015-09-02')],
    'HH': [('2015-01-01','2015-01-06'), ('2015-01-30','2015-01-30'), ('2015-03-02','2015-03-13'), ('2015-05-11','2015-05-15'), ('2015-07-16','2015-08-26')],
    'HE': [('2015-01-01','2015-01-10'), ('2015-03-30','2015-04-11'), ('2015-07-27','2015-09-04')],
    'MV': [('2015-01-01','2015-01-02'), ('2015-02-02','2015-02-14'), ('2015-03-30','2015-04-08'), ('2015-05-22','2015-05-26'), ('2015-07-20','2015-08-29')],
    'NI': [('2015-01-01','2015-01-05'), ('2015-02-02','2015-02-03'), ('2015-03-25','2015-04-10'), ('2015-05-15','2015-05-15'), ('2015-05-26','2015-05-26'), ('2015-07-23','2015-09-02')],
    'NW': [('2015-01-01','2015-01-06'), ('2015-03-30','2015-04-11'), ('2015-05-26','2015-05-26'), ('2015-06-29','2015-08-11')],
    'RP': [('2015-01-01','2015-01-07'), ('2015-03-26','2015-04-10'), ('2015-05-15','2015-05-15'), ('2015-06-05','2015-06-05'), ('2015-07-27','2015-09-04')],
    'SL': [('2015-01-01','2015-01-07'), ('2015-02-16','2015-02-21'), ('2015-03-30','2015-04-11'), ('2015-07-27','2015-09-05')],
    'SN': [('2015-01-01','2015-01-03'), ('2015-02-09','2015-02-21'), ('2015-04-02','2015-04-11'), ('2015-05-15','2015-05-15'), ('2015-07-13','2015-08-21')],
    'ST': [('2015-01-01','2015-01-05'), ('2015-02-02','2015-02-14'), ('2015-04-02','2015-04-02'), ('2015-05-15','2015-05-23'), ('2015-07-13','2015-08-26')],
    'SH': [('2015-01-01','2015-01-06'), ('2015-04-01','2015-04-17'), ('2015-05-15','2015-05-15'), ('2015-07-20','2015-08-29')],
    'TH': [('2015-01-01','2015-01-03'), ('2015-02-02','2015-02-07'), ('2015-03-30','2015-04-11'), ('2015-05-15','2015-05-15'), ('2015-07-13','2015-08-21')],
}
DE_hol_2015_2_dict = {
    'BW': [('2015-11-02', '2015-11-06'), ('2015-12-23', '2015-12-31')],
    'BY': [('2015-11-02', '2015-11-07'), ('2015-12-24', '2015-12-31')],
    'BE': [('2015-10-19', '2015-10-31'), ('2015-12-23', '2015-12-31')],
    'BB': [('2015-10-19', '2015-10-30'), ('2015-12-23', '2015-12-31')],
    'HB': [('2015-10-19', '2015-10-31'), ('2015-12-23', '2015-12-31')],
    'HH': [('2015-10-19', '2015-10-30'), ('2015-12-23', '2015-12-31')],
    'HE': [('2015-10-19', '2015-10-31'), ('2015-12-23', '2015-12-31')],
    'MV': [('2015-10-24', '2015-10-30'), ('2015-12-23', '2015-12-31')],
    'NI': [('2015-10-19', '2015-10-31'), ('2015-12-23', '2015-12-31')],
    'NW': [('2015-10-05', '2015-10-17'), ('2015-12-23', '2015-12-31')],
    'RP': [('2015-10-19', '2015-10-30'), ('2015-12-23', '2015-12-31')],
    'SL': [('2015-10-19', '2015-10-31'), ('2015-12-23', '2015-12-31')],
    'SN': [('2015-10-12', '2015-10-24'), ('2015-12-23', '2015-12-31')],
    'ST': [('2015-10-17', '2015-10-24'), ('2015-12-23', '2015-12-31')],
    'SH': [('2015-10-19', '2015-10-31'), ('2015-12-23', '2015-12-31')],
    'TH': [('2015-10-05', '2015-10-17'), ('2015-12-23', '2015-12-31')],
}
DE_hol_2016_1_dict = {
    'BW': [('2016-01-01', '2016-01-09'), ('2016-03-29', '2016-04-02'),
           ('2016-05-17', '2016-05-28'), ('2016-07-28', '2016-09-10')],
    'BY': [('2016-01-01', '2016-01-05'), ('2016-02-08', '2016-02-12'),
           ('2016-03-21', '2016-04-01'), ('2016-05-17', '2016-05-28'),
           ('2016-07-30', '2016-09-12')],
    'BE': [('2016-01-01', '2016-01-02'), ('2016-02-01', '2016-02-06'),
           ('2016-03-21', '2016-04-02'), ('2016-05-06', '2016-05-06'),
           ('2016-05-17', '2016-05-17'), ('2016-07-21', '2016-09-02')],
    'BB': [('2016-01-01', '2016-01-02'), ('2016-02-01', '2016-02-06'),
           ('2016-03-23', '2016-04-02'), ('2016-05-06', '2016-05-06'),
           ('2016-05-17', '2016-05-17'), ('2016-07-21', '2016-09-03')],
    'HB': [('2016-01-01', '2016-01-06'), ('2016-01-28', '2016-01-29'),
           ('2016-03-18', '2016-04-02'), ('2016-05-17', '2016-05-17'),
           ('2016-06-23', '2016-08-03')],
    'HH': [('2016-01-01', '2016-01-01'), ('2016-01-29', '2016-01-29'),
           ('2016-03-07', '2016-03-18'), ('2016-05-06', '2016-05-06'),
           ('2016-05-17', '2016-05-20'), ('2016-07-21', '2016-08-31')],
    'HE': [('2016-01-01', '2016-01-09'), ('2016-03-29', '2016-04-09'),
           ('2016-07-18', '2016-08-26')],
    'MV': [('2016-01-01', '2016-01-02'), ('2016-02-01', '2016-02-13'),
           ('2016-03-21', '2016-03-30'), ('2016-05-14', '2016-05-17'),
           ('2016-07-25', '2016-09-03')],
    'NI': [('2016-01-01', '2016-01-06'), ('2016-01-28', '2016-01-29'),
           ('2016-03-18', '2016-04-02'), ('2016-05-06', '2016-05-06'),
           ('2016-05-17', '2016-05-17'), ('2016-06-23', '2016-08-03')],
    'NW': [('2016-01-01', '2016-01-06'), ('2016-03-21', '2016-04-02'),
           ('2016-05-17', '2016-05-17'), ('2016-07-11', '2016-08-23')],
    'RP': [('2016-01-01', '2016-01-08'), ('2016-03-18', '2016-04-01'),
           ('2016-05-06', '2016-05-06'), ('2016-05-27', '2016-05-27'),
           ('2016-07-18', '2016-08-26')],
    'SL': [('2016-01-01', '2016-01-02'), ('2016-02-08', '2016-02-13'),
           ('2016-03-29', '2016-04-09'), ('2016-07-18', '2016-08-27')],
    'SN': [('2016-01-01', '2016-01-02'), ('2016-02-08', '2016-02-20'),
           ('2016-03-25', '2016-04-02'), ('2016-05-06', '2016-05-06'),
           ('2016-06-27', '2016-08-05')],
    'ST': [('2016-01-01', '2016-01-05'), ('2016-02-01', '2016-02-10'),
           ('2016-03-24', '2016-03-24'), ('2016-05-06', '2016-05-14'),
           ('2016-06-27', '2016-08-10')],
    'SH': [('2016-01-01', '2016-01-06'), ('2016-03-24', '2016-04-09'),
           ('2016-05-06', '2016-05-06'), ('2016-07-25', '2016-09-03')],
    'TH': [('2016-01-01', '2016-01-02'), ('2016-02-01', '2016-02-06'),
           ('2016-03-24', '2016-04-02'), ('2016-05-06', '2016-05-06'),
           ('2016-06-27', '2016-08-10')],
}
DE_hol_2016_2_dict = {
    'BW': [('2016-11-02', '2016-11-04'), ('2016-12-23', '2016-12-31')],
    'BY': [('2016-10-31', '2016-11-04'), ('2016-12-24', '2016-12-31')],
    'BE': [('2016-10-17', '2016-10-28'), ('2016-12-23', '2016-12-31')],
    'BB': [('2016-10-17', '2016-10-28'), ('2016-12-23', '2016-12-31')],
    'HB': [('2016-10-04', '2016-10-15'), ('2016-12-21', '2016-12-31')],
    'HH': [('2016-10-17', '2016-10-29'), ('2016-12-27', '2016-12-31')],
    'HE': [('2016-10-17', '2016-10-29'), ('2016-12-22', '2016-12-31')],
    'MV': [('2016-10-24', '2016-10-29'), ('2016-12-22', '2016-12-31')],
    'NI': [('2016-10-04', '2016-10-15'), ('2016-12-21', '2016-12-31')],
    'NW': [('2016-10-10', '2016-10-21'), ('2016-12-23', '2016-12-31')],
    'RP': [('2016-10-10', '2016-10-21'), ('2016-12-22', '2016-12-31')],
    'SL': [('2016-10-10', '2016-10-22'), ('2016-12-19', '2016-12-31')],
    'SN': [('2016-10-03', '2016-10-15'), ('2016-12-23', '2016-12-31')],
    'ST': [('2016-10-04', '2016-10-15'), ('2016-12-19', '2016-12-31')],
    'SH': [('2016-10-17', '2016-10-29'), ('2016-12-23', '2016-12-31')],
    'TH': [('2016-10-10', '2016-10-22'), ('2016-12-23', '2016-12-31')],
}
DE_hol_2017_1_dict = {
    'BW': [('2017-01-01', '2017-01-07'), ('2017-04-10', '2017-04-21'), 
           ('2017-06-06', '2017-06-16'), ('2017-07-27', '2017-09-09')],
    'BY': [('2017-01-01', '2017-01-05'), ('2017-02-27', '2017-03-03'), 
           ('2017-04-10', '2017-04-22'), ('2017-06-06', '2017-06-16'), 
           ('2017-07-29', '2017-09-11')],
    'BE': [('2017-01-01', '2017-01-03'), ('2017-01-30', '2017-02-04'), 
           ('2017-04-10', '2017-04-17'), ('2017-06-06', '2017-06-09'), 
           ('2017-07-20', '2017-09-01')],
    'BB': [('2017-01-01', '2017-01-03'), ('2017-01-30', '2017-02-04'), 
           ('2017-04-12', '2017-04-22'), ('2017-05-26', '2017-05-26'), 
           ('2017-07-20', '2017-09-01')],
    'HB': [('2017-01-01', '2017-01-06'), ('2017-01-30', '2017-01-31'), 
           ('2017-04-10', '2017-04-22'), ('2017-06-06', '2017-06-06'), 
           ('2017-07-22', '2017-09-02')],
    'HH': [('2017-01-01', '2017-01-06'), 
           ('2017-01-30', '2017-01-30'), ('2017-03-06', '2017-03-17'), 
           ('2017-05-22', '2017-05-26'), ('2017-07-20', '2017-08-30')],
    'HE': [('2017-01-01', '2017-01-07'), ('2017-04-03', '2017-04-15'), 
           ('2017-07-03', '2017-08-11')],
    'MV': [('2017-01-01', '2017-01-02'), ('2017-02-06', '2017-02-18'), 
           ('2017-04-10', '2017-04-19'), ('2017-06-02', '2017-06-06'), 
           ('2017-07-24', '2017-09-02')],
    'NI': [('2017-01-01', '2017-01-06'), ('2017-01-30', '2017-01-31'), 
           ('2017-04-10', '2017-04-22'),  ('2017-05-26', '2017-05-26'), 
           ('2017-06-06','2017-06-06'), ('2017-06-22', '2017-08-02')],
    ## two single day intervals in the pentecost period
    'NW': [('2017-01-01', '2017-01-06'), ('2017-04-10', '2017-04-22'), 
           ('2017-06-06', '2017-06-06'), ('2017-07-17', '2017-08-29')],
    'RP': [('2017-01-01', '2017-01-06'), ('2017-04-10', '2017-04-21'), 
           ('2017-05-26', '2017-05-26'),  ('2017-06-16','2017-06-16'), 
           ('2017-07-03', '2017-08-11')],
    ## `RP` has two single day intervals in the pentecost period. one atypical date: '2017-06-16'
    'SL': [('2017-01-01', '2017-01-01'), ('2017-02-27', '2017-03-04'), 
           ('2017-04-10', '2017-04-22'), ('2017-07-03', '2017-08-14')],
    ## Saarland, `SL` , has no school holidays in the beginning of January, for readability I left the New Year holiday as one-day interval
    'SN': [('2017-01-01', '2017-01-02'), ('2017-02-13', '2017-02-24'), 
           ('2017-04-13', '2017-04-22'), ('2017-05-26', '2017-05-26'), 
           ('2017-06-26', '2017-08-04')],
    ## Sachsen has a 26-th of may as single day, as all the remaining states.
    'ST': [('2017-01-01', '2017-01-02'), ('2017-02-04', '2017-02-11'), 
           ('2017-04-10', '2017-04-13'), ('2017-05-26', '2017-05-26'), 
           ('2017-06-26', '2017-08-09')],
    'SH': [('2017-01-01', '2017-01-06'), ('2017-04-07', '2017-04-21'), 
           ('2017-05-26', '2017-05-26'), ('2017-07-24', '2017-09-02')],
    'TH': [('2017-01-01', '2017-01-01'), ('2017-02-06', '2017-02-11'), 
           ('2017-04-10', '2017-04-21'), ('2017-05-26', '2017-05-26'), 
           ('2017-06-26', '2017-08-09')],
}
DE_hol_2017_2_dict = {
    'BW': [('2017-10-30', '2017-11-03'), ('2017-12-22', '2017-12-31')],
    'BY': [('2017-10-30', '2017-11-03'), ('2017-12-23', '2017-12-31')],
    'BE': [('2017-10-02', '2017-10-02'), ('2017-10-23', '2017-11-04'), 
           ('2017-12-21', '2017-12-31')],
    'BB': [('2017-10-23', '2017-11-04'), ('2017-12-21', '2017-12-31')],
    'HB': [('2017-10-02', '2017-10-14'), ('2017-10-30', '2017-10-30'), 
           ('2017-12-22', '2017-12-31')],
    'HH': [('2017-10-02', '2017-10-02'), ('2017-10-16', '2017-10-27'), 
           ('2017-12-22', '2017-12-31')],
    'HE': [('2017-10-09', '2017-10-21'), ('2017-12-24', '2017-12-31')],
    'MV': [('2017-10-23', '2017-10-28'), ('2017-12-21', '2017-12-31')],
    'NI': [('2017-10-02', '2017-10-13'), ('2017-10-30', '2017-10-30'), 
           ('2017-12-22', '2017-12-31')],
    'NW': [('2017-10-23', '2017-11-04'), ('2017-12-27', '2017-12-31')],
    'RP': [('2017-10-02', '2017-10-13'), ('2017-12-22', '2017-12-31')],
    'SL': [('2017-10-02', '2017-10-14'), ('2017-12-21', '2017-12-31')],
    'SN': [('2017-10-02', '2017-10-14'), ('2017-10-30', '2017-10-30'), 
           ('2017-12-23', '2017-12-31')],
    'ST': [('2017-10-02', '2017-10-13'), ('2017-10-30', '2017-10-30'), 
           ('2017-12-21', '2017-12-31')],
    'SH': [('2017-10-16', '2017-10-27'), ('2017-12-21', '2017-12-31')],
    'TH': [('2017-10-02', '2017-10-14'), ('2017-12-22', '2017-12-31')],
}
DE_hol_2018_1_dict = {
    'BW': [('2018-01-01', '2018-01-05'), ('2018-03-26', '2018-04-06'),
           ('2018-05-22', '2018-06-02'), ('2018-07-26', '2018-09-08')],
    'BY': [('2018-01-01', '2018-01-05'), ('2018-02-12', '2018-02-16'),
           ('2018-03-26', '2018-04-07'), ('2018-05-22', '2018-06-02'),
           ('2018-07-30', '2018-09-10')],
    'BE': [('2018-01-01', '2018-01-02'), ('2018-02-05', '2018-02-10'),
           ('2018-03-26', '2018-04-06'), ('2018-04-30', '2018-04-30'),
           ('2018-05-11', '2018-05-11'), ('2018-05-22', '2018-05-22'),
           ('2018-07-05', '2018-08-17')],
    'BB': [('2018-01-01', '2018-01-02'), ('2018-02-05', '2018-02-10'),
           ('2018-03-26', '2018-04-06'), ('2018-07-05', '2018-08-18')],
    'HB': [('2018-01-01', '2018-01-06'), ('2018-02-01', '2018-02-02'),
           ('2018-03-19', '2018-04-03'), ('2018-04-30', '2018-04-30'),
           ('2018-05-11', '2018-05-11'), ('2018-05-22', '2018-05-22'),
           ('2018-06-28', '2018-08-08')],
    'HH': [('2018-01-01', '2018-01-05'), ('2018-02-02', '2018-02-02'),
           ('2018-03-05', '2018-03-16'), ('2018-04-30', '2018-04-30'),
           ('2018-05-07', '2018-05-11'), ('2018-07-05', '2018-08-15')],
    'HE': [('2018-01-01', '2018-01-13'), ('2018-03-26', '2018-04-07'),
           ('2018-06-25', '2018-08-03')],
    'MV': [('2018-01-01', '2018-01-03'), ('2018-02-05', '2018-02-16'),
           ('2018-03-26', '2018-04-04'), ('2018-05-18', '2018-05-22'),
           ('2018-07-09', '2018-08-18')],
    'NI': [('2018-01-01', '2018-01-05'), ('2018-02-01', '2018-02-02'),
           ('2018-03-19', '2018-04-03'), ('2018-04-30', '2018-04-30'),
           ('2018-05-11', '2018-05-11'), ('2018-05-22', '2018-05-22'),
           ('2018-06-28', '2018-08-08')],
    'NW': [('2018-01-01', '2018-01-06'), ('2018-03-26', '2018-04-07'),
           ('2018-05-22', '2018-05-25'), ('2018-07-16', '2018-08-28')],
    'RP': [('2018-01-01', '2018-01-09'), ('2018-03-26', '2018-04-06'),
           ('2018-06-25', '2018-08-03')],
    'SL': [('2018-01-01', '2018-01-05'), ('2018-02-12', '2018-02-17'),
           ('2018-03-26', '2018-04-06'), ('2018-06-25', '2018-08-03')],
    'SN': [('2018-01-01', '2018-01-02'), ('2018-02-12', '2018-02-23'),
           ('2018-03-29', '2018-04-06'), ('2018-05-11', '2018-05-11'),
           ('2018-05-19', '2018-05-22'), ('2018-07-02', '2018-08-10')],
    'ST': [('2018-01-01', '2018-01-03'), ('2018-02-05', '2018-02-09'),
           ('2018-03-26', '2018-03-31'), ('2018-05-11', '2018-05-18'),
           ('2018-06-28', '2018-08-08')],
    'SH': [('2018-01-01', '2018-01-06'), ('2018-03-29', '2018-04-13'),
           ('2018-05-11', '2018-05-11'), ('2018-07-09', '2018-08-18')],
    'TH': [('2018-01-01', '2018-01-05'), ('2018-02-05', '2018-02-09'),
           ('2018-03-26', '2018-04-07'), ('2018-05-11', '2018-05-11'),
           ('2018-07-02', '2018-08-11')],
}
DE_hol_2018_2_dict = {
    'BW': [('2018-10-29', '2018-11-02'), ('2018-12-24', '2018-12-31')],
    'BY': [('2018-10-29', '2018-11-02'), ('2018-12-22', '2018-12-31')],
    'BE': [('2018-10-22', '2018-11-02'), ('2018-12-22', '2018-12-31')],
    'BB': [('2018-10-22', '2018-11-02'), ('2018-12-22', '2018-12-31')],
    'HB': [('2018-10-01', '2018-10-13'), ('2018-12-24', '2018-12-31')],
    'HH': [('2018-10-01', '2018-10-12'), ('2018-12-20', '2018-12-31')],
    'HE': [('2018-10-01', '2018-10-13'), ('2018-12-24', '2018-12-31')],
    'MV': [('2018-10-08', '2018-10-13'), ('2018-12-24', '2018-12-31')],
    'NI': [('2018-10-01', '2018-10-12'), ('2018-12-24', '2018-12-31')],
    'NW': [('2018-10-15', '2018-10-27'), ('2018-12-21', '2018-12-31')],
    'RP': [('2018-10-01', '2018-10-12'), ('2018-12-20', '2018-12-31')],
    'SL': [('2018-10-01', '2018-10-12'), ('2018-12-20', '2018-12-31')],
    'SN': [('2018-10-08', '2018-10-20'), ('2018-12-22', '2018-12-31')],
    'ST': [('2018-10-01', '2018-10-12'), ('2018-12-19', '2018-12-31')],
    'SH': [('2018-10-01', '2018-10-19'), ('2018-12-21', '2018-12-31')],
    'TH': [('2018-10-01', '2018-10-13'), ('2018-12-21', '2018-12-31')],
}
DE_hol_2019_1_dict = {
    'BW': [('2019-01-01', '2019-01-05'), ('2019-04-15', '2019-04-27'),
           ('2019-06-11', '2019-06-21'), ('2019-07-29', '2019-09-10')],
    'BY': [('2019-01-01', '2019-01-05'), ('2019-03-04', '2019-03-08'),
           ('2019-04-15', '2019-04-27'), ('2019-06-11', '2019-06-21'),
           ('2019-07-29', '2019-09-09')],
    'BE': [('2019-01-01', '2019-01-05'), ('2019-02-04', '2019-02-09'),
           ('2019-04-15', '2019-04-26'), ('2019-05-31', '2019-05-31'),
           ('2019-06-11', '2019-06-11'), ('2019-06-20', '2019-08-02')],
    'BB': [('2019-01-01', '2019-01-05'), ('2019-02-04', '2019-02-09'),
           ('2019-04-15', '2019-04-26'), ('2019-06-20', '2019-08-03')],
    'HB': [('2019-01-01', '2019-01-04'), ('2019-01-31', '2019-02-01'),
           ('2019-04-06', '2019-04-23'), ('2019-05-31', '2019-05-31'),
           ('2019-06-11', '2019-06-11'), ('2019-07-04', '2019-08-14')],
    'HH': [('2019-01-01', '2019-01-04'), ('2019-02-01', '2019-02-01'),
           ('2019-03-04', '2019-03-15'), ('2019-05-13', '2019-05-17'),
           ('2019-05-31', '2019-05-31'), ('2019-06-27', '2019-08-07')],
    'HE': [('2019-01-01', '2019-01-12'), ('2019-04-15', '2019-04-27'),
           ('2019-07-01', '2019-08-09')],
    'MV': [('2019-01-01', '2019-01-05'), ('2019-02-04', '2019-02-15'),
           ('2019-04-15', '2019-04-24'), ('2019-06-07', '2019-06-11'),
           ('2019-07-01', '2019-08-10')],
    'NI': [('2019-01-01', '2019-01-04'), ('2019-01-31', '2019-02-01'),
           ('2019-04-08', '2019-04-23'), ('2019-05-31', '2019-05-31'),
           ('2019-06-11', '2019-06-11'), ('2019-07-04', '2019-08-14')],
    'NW': [('2019-01-01', '2019-01-04'), ('2019-04-15', '2019-04-27'),
           ('2019-06-11', '2019-06-11'), ('2019-07-15', '2019-08-27')],
    'RP': [('2019-01-01', '2019-01-04'), ('2019-02-25', '2019-03-01'),
           ('2019-04-23', '2019-04-30'), ('2019-07-01', '2019-08-09')],
    'SL': [('2019-01-01', '2019-01-04'), ('2019-02-25', '2019-03-05'),
           ('2019-04-17', '2019-04-26'), ('2019-07-01', '2019-08-09')],
    'SN': [('2019-01-01', '2019-01-04'), ('2019-02-18', '2019-03-02'),
           ('2019-04-19', '2019-04-26'), ('2019-05-31', '2019-05-31'),
           ('2019-07-08', '2019-08-16')],
    'ST': [('2019-01-01', '2019-01-04'), ('2019-02-11', '2019-02-15'),
           ('2019-04-18', '2019-04-30'), ('2019-05-31', '2019-06-01'),
           ('2019-07-04', '2019-08-14')],
    'SH': [('2019-01-01', '2019-01-04'), ('2019-04-04', '2019-04-18'),
           ('2019-05-31', '2019-05-31'), ('2019-07-01', '2019-08-10')],
    'TH': [('2019-01-01', '2019-01-04'), ('2019-02-11', '2019-02-15'),
           ('2019-04-15', '2019-04-27'), ('2019-05-31', '2019-05-31'),
           ('2019-07-08', '2019-08-17')],
}
DE_hol_2019_2_dict = {
    'BW': [('2019-10-28', '2019-10-31'), ('2019-12-23', '2019-12-31')],
    'BY': [('2019-10-28', '2019-10-31'), ('2019-12-23', '2019-12-31')],
    'BE': [('2019-10-04', '2019-10-04'), ('2019-10-07', '2019-10-19'), ('2019-12-23', '2019-12-31')],
    'BB': [('2019-10-04', '2019-10-18'), ('2019-12-23', '2019-12-31')],
    'HB': [('2019-10-04', '2019-10-18'), ('2019-12-21', '2019-12-31')],
    'HH': [('2019-10-04', '2019-10-18'), ('2019-12-23', '2019-12-31')],
    'HE': [('2019-09-30', '2019-10-12'), ('2019-12-23', '2019-12-31')],
    'MV': [('2019-10-07', '2019-10-12'), ('2019-12-23', '2019-12-31')],
    'NI': [('2019-10-04', '2019-10-18'), ('2019-12-23', '2019-12-31')],
    'NW': [('2019-10-14', '2019-10-26'), ('2019-12-23', '2019-12-31')],
    'RP': [('2019-09-30', '2019-10-11'), ('2019-12-23', '2019-12-31')],
    'SL': [('2019-10-07', '2019-10-18'), ('2019-12-23', '2019-12-31')],
    'SN': [('2019-10-14', '2019-10-25'), ('2019-12-21', '2019-12-31')],
    'ST': [('2019-10-04', '2019-10-11'), ('2019-11-01', '2019-11-01'), 
           ('2019-12-23', '2019-12-31')],
    'SH': [('2019-10-04', '2019-10-18'), ('2019-12-23', '2019-12-31')],
    'TH': [('2019-10-07', '2019-10-19'), ('2019-12-21', '2019-12-31')],
}
DE_hol_2020_1_dict = {
    'BW': [('2020-01-01', '2020-01-04'), ('2020-04-06', '2020-04-18'),
           ('2020-06-02', '2020-06-13'), ('2020-07-30', '2020-09-12')],
    'BY': [('2020-01-01', '2020-01-04'), ('2020-02-24', '2020-02-28'),
           ('2020-04-06', '2020-04-18'), ('2020-06-02', '2020-06-13'),
           ('2020-07-27', '2020-09-07')],
    'BE': [('2020-01-01', '2020-01-04'), ('2020-02-03', '2020-02-08'),
           ('2020-04-06', '2020-04-17'), ('2020-05-22', '2020-05-22'),
           ('2020-06-25', '2020-08-07')],
    'BB': [('2020-01-01', '2020-01-03'), ('2020-02-03', '2020-02-08'),
           ('2020-04-06', '2020-04-17'), ('2020-06-25', '2020-08-08')],
    'HB': [('2020-01-01', '2020-01-06'), ('2020-02-03', '2020-02-04'),
           ('2020-03-28', '2020-04-14'), ('2020-05-22', '2020-05-22'),
           ('2020-06-02', '2020-06-02'), ('2020-07-16', '2020-08-26')],
    'HH': [('2020-01-01', '2020-01-03'), ('2020-01-31', '2020-01-31'),
           ('2020-03-02', '2020-03-13'), ('2020-05-04', '2020-05-08'),
           ('2020-05-22', '2020-05-22'), ('2020-06-25', '2020-08-05')],
    'HE': [('2020-01-01', '2020-01-11'), ('2020-04-06', '2020-04-18'),
           ('2020-07-06', '2020-08-14')],
    'MV': [('2020-01-01', '2020-01-04'), ('2020-02-10', '2020-02-21'),
           ('2020-04-06', '2020-04-15'), ('2020-05-29', '2020-06-02'),
           ('2020-06-22', '2020-08-01')],
    'NI': [('2020-01-01', '2020-01-06'), ('2020-02-03', '2020-02-04'),
           ('2020-03-30', '2020-04-14'), ('2020-05-22', '2020-05-22'),
           ('2020-06-02', '2020-06-02'), ('2020-07-16', '2020-08-26')],
    'NW': [('2020-01-01', '2020-01-06'), ('2020-04-06', '2020-04-18'),
           ('2020-06-02', '2020-06-02'), ('2020-06-29', '2020-08-11')],
    'RP': [('2020-01-01', '2020-01-06'), ('2020-02-17', '2020-02-21'),
           ('2020-04-09', '2020-04-17'), ('2020-07-06', '2020-08-14')],
    'SL': [('2020-01-01', '2020-01-03'), ('2020-02-17', '2020-02-25'),
           ('2020-04-14', '2020-04-24'), ('2020-07-06', '2020-08-14')],
    'SN': [('2020-01-01', '2020-01-03'), ('2020-02-10', '2020-02-22'),
           ('2020-04-10', '2020-04-18'), ('2020-05-22', '2020-05-22'),
           ('2020-07-20', '2020-08-28')],
    'ST': [('2020-01-01', '2020-01-04'), ('2020-02-10', '2020-02-14'),
           ('2020-04-06', '2020-04-11'), ('2020-05-18', '2020-05-30'),
           ('2020-07-16', '2020-08-26')],
    'SH': [('2020-01-01', '2020-01-06'), ('2020-03-30', '2020-04-17'),
           ('2020-05-22', '2020-05-22'), ('2020-06-29', '2020-08-08')],
    'TH': [('2020-01-01', '2020-01-03'), ('2020-02-10', '2020-02-14'),
           ('2020-04-06', '2020-04-18'), ('2020-05-22', '2020-05-22'),
           ('2020-07-20', '2020-08-29')],
}
DE_hol_2020_2_dict = {
    'BW': [('2020-10-26', '2020-10-31'), ('2020-12-23', '2020-12-31')],
    'BY': [('2020-10-31', '2020-11-06'), ('2020-12-23', '2020-12-31')],
    'BE': [('2020-10-12', '2020-10-24'), ('2020-12-21', '2020-12-31')],
    'BB': [('2020-10-12', '2020-10-24'), ('2020-12-21', '2020-12-31')],
    'HB': [('2020-10-12', '2020-10-24'), ('2020-12-23', '2020-12-31')],
    'HH': [('2020-10-05', '2020-10-16'), ('2020-12-21', '2020-12-31')],
    'HE': [('2020-10-05', '2020-10-17'), ('2020-12-21', '2020-12-31')],
    'MV': [('2020-10-05', '2020-10-10'), ('2020-11-02', '2020-11-03'), 
           ('2020-12-21', '2020-12-31')],
    'NI': [('2020-10-12', '2020-10-23'), ('2020-12-23', '2020-12-31')],
    'NW': [('2020-10-12', '2020-10-24'), ('2020-12-23', '2020-12-31')],
    'RP': [('2020-10-12', '2020-10-23'), ('2020-12-21', '2020-12-31')],
    'SL': [('2020-10-12', '2020-10-23'), ('2020-12-21', '2020-12-31')],
    'SN': [('2020-10-19', '2020-10-31'), ('2020-12-23', '2020-12-31')],
    'ST': [('2020-10-19', '2020-10-24'), ('2020-12-21', '2020-12-31')],
    'SH': [('2020-10-05', '2020-10-17'), ('2020-12-21', '2020-12-31')],
    'TH': [('2020-10-17', '2020-10-30'), ('2020-12-23', '2020-12-31')],}

In [ ]:
DE_hol_2021_1_dict = {
    'BW': [('2021-10-31', '2021-11-06'), ('2021-12-23', '2021-12-31')],
    'BY': [('2021-11-02', '2021-11-05'), ('2021-12-24', '2021-12-31')],
    'BE': [('2021-10-11', '2021-10-23'), ('2021-12-23', '2021-12-31')],
    'BB': [('2021-10-11', '2021-10-23'), ('2021-12-23', '2021-12-31')],
    'HB': [('2021-10-18', '2021-10-30'), ('2021-12-23', '2021-12-31')],
    'HH': [('2021-10-04', '2021-10-15'), ('2021-12-23', '2021-12-31')],
    'HE': [('2021-10-11', '2021-10-23'), ('2021-12-23', '2021-12-31')],
    'MV': [('2021-10-02', '2021-10-09'), ('2021-11-01', '2021-11-02'), ('2021-12-22', '2021-12-31')],
    'NI': [('2021-10-18', '2021-10-29'), ('2021-12-23', '2021-12-31')],
    'NW': [('2021-10-11', '2021-10-23'), ('2021-12-24', '2021-12-31')],
    'RP': [('2021-10-11', '2021-10-22'), ('2021-12-23', '2021-12-31')],
    'SL': [('2021-10-18', '2021-10-29'), ('2021-12-23', '2021-12-31')],
    'SN': [('2021-10-18', '2021-10-30'), ('2021-12-23', '2021-12-31')],
    'ST': [('2021-10-25', '2021-10-30'), ('2021-12-22', '2021-12-31')],
    'SH': [('2021-10-04', '2021-10-16'), ('2021-12-23', '2021-12-31')],
    'TH': [('2021-10-25', '2021-11-06'), ('2021-12-23', '2021-12-31')],
}
DE_hol_2021_2_dict = {
    'BW': [('2021-10-31', '2021-11-06'), ('2021-12-23', '2021-12-31')],
    'BY': [('2021-11-02', '2021-11-05'), ('2021-12-24', '2021-12-31')],
    'BE': [('2021-10-11', '2021-10-23'), ('2021-12-23', '2021-12-31')],
    'BB': [('2021-10-11', '2021-10-23'), ('2021-12-23', '2021-12-31')],
    'HB': [('2021-10-18', '2021-10-30'), ('2021-12-23', '2021-12-31')],
    'HH': [('2021-10-04', '2021-10-15'), ('2021-12-23', '2021-12-31')],
    'HE': [('2021-10-11', '2021-10-23'), ('2021-12-23', '2021-12-31')],
    'MV': [('2021-10-02', '2021-10-09'), ('2021-11-01', '2021-11-02'), ('2021-12-22', '2021-12-31')],
    'NI': [('2021-10-18', '2021-10-29'), ('2021-12-23', '2021-12-31')],
    'NW': [('2021-10-11', '2021-10-23'), ('2021-12-24', '2021-12-31')],
    'RP': [('2021-10-11', '2021-10-22'), ('2021-12-23', '2021-12-31')],
    'SL': [('2021-10-18', '2021-10-29'), ('2021-12-23', '2021-12-31')],
    'SN': [('2021-10-18', '2021-10-30'), ('2021-12-23', '2021-12-31')],
    'ST': [('2021-10-25', '2021-10-30'), ('2021-12-22', '2021-12-31')],
    'SH': [('2021-10-04', '2021-10-16'), ('2021-12-23', '2021-12-31')],
    'TH': [('2021-10-25', '2021-11-06'), ('2021-12-23', '2021-12-31')],
}
DE_hol_2022_1_dict = {
    'BW': [('2022-01-01', '2022-01-08'), ('2022-04-14', '2022-04-23'),
           ('2022-06-07', '2022-06-18'), ('2022-07-28', '2022-09-10')],
    'BY': [('2022-01-01', '2022-01-08'), ('2022-02-28', '2022-03-04'),
           ('2022-04-11', '2022-04-23'), ('2022-06-07', '2022-06-18'),
           ('2022-08-01', '2022-09-12')],
    'BE': [('2022-01-01', '2022-01-01'), ('2022-01-29', '2022-02-05'),
           ('2022-04-11', '2022-04-23'), ('2022-05-27', '2022-05-27'),
           ('2022-06-07', '2022-06-07'), ('2022-07-07', '2022-08-19')],
    'BB': [('2022-01-01', '2022-01-01'), ('2022-01-31', '2022-02-05'),
           ('2022-04-11', '2022-04-23'), ('2022-07-07', '2022-08-20')],
    'HB': [('2022-01-01', '2022-01-08'), ('2022-01-31', '2022-02-01'),
           ('2022-04-04', '2022-04-19'), ('2022-05-27', '2022-05-27'),
           ('2022-06-07', '2022-06-07'), ('2022-07-14', '2022-08-24')],
    'HH': [('2022-01-01', '2022-01-04'), ('2022-01-28', '2022-01-28'),
           ('2022-03-07', '2022-03-18'), ('2022-05-23', '2022-05-27'),
           ('2022-07-07', '2022-08-17')],
    'HE': [('2022-01-01', '2022-01-08'), ('2022-04-11', '2022-04-23'),
           ('2022-07-25', '2022-09-02')],
    'MV': [('2022-01-01', '2022-01-01'), ('2022-02-05', '2022-02-18'),
           ('2022-04-11', '2022-04-20'), ('2022-05-27', '2022-05-27'),
           ('2022-06-03', '2022-06-07'), ('2022-07-04', '2022-08-13')],
    'NI': [('2022-01-01', '2022-01-07'), ('2022-01-31', '2022-02-01'),
           ('2022-04-04', '2022-04-19'), ('2022-05-27', '2022-05-27'),
           ('2022-06-07', '2022-06-07'), ('2022-07-14', '2022-08-24')],
    'NW': [('2022-01-01', '2022-01-08'), ('2022-04-11', '2022-04-23'),
           ('2022-06-27', '2022-08-09')],
    'RP': [('2022-01-01', '2022-01-01'), ('2022-02-21', '2022-02-25'),
           ('2022-04-13', '2022-04-22'), ('2022-07-25', '2022-09-02')],
    'SL': [('2022-01-01', '2022-01-03'), ('2022-02-21', '2022-03-01'),
           ('2022-04-14', '2022-04-22'), ('2022-06-07', '2022-06-10'),
           ('2022-07-25', '2022-09-02')],
    'SN': [('2022-01-01', '2022-01-01'), ('2022-02-12', '2022-02-26'),
           ('2022-04-15', '2022-04-23'), ('2022-05-27', '2022-05-27'),
           ('2022-07-18', '2022-08-26')],
    'ST': [('2022-01-01', '2022-01-08'), ('2022-02-12', '2022-02-19'),
           ('2022-04-11', '2022-04-16'), ('2022-05-23', '2022-05-28'),
           ('2022-07-14', '2022-08-24')],
    'SH': [('2022-01-01', '2022-01-08'), ('2022-04-04', '2022-04-16'),
           ('2022-05-27', '2022-05-28'), ('2022-07-04', '2022-08-13')],
    'TH': [('2022-01-01', '2022-01-01'), ('2022-02-12', '2022-02-19'),
           ('2022-04-11', '2022-04-23'), ('2022-05-27', '2022-05-27'),
           ('2022-07-18', '2022-08-27')],
}
DE_hol_2022_2_dict = {
    'BW': [('2022-10-31', '2022-10-31'), ('2022-11-02', '2022-11-04'), ('2022-12-21', '2022-12-31')],
    'BY': [('2022-10-31', '2022-11-04'), ('2022-12-24', '2022-12-31')],
    'BE': [('2022-10-24', '2022-11-05'), ('2022-12-22', '2022-12-31')],
    'BB': [('2022-10-24', '2022-11-05'), ('2022-12-22', '2022-12-31')],
    'HB': [('2022-10-17', '2022-10-29'), ('2022-12-23', '2022-12-31')],
    'HH': [('2022-10-10', '2022-10-21'), ('2022-12-23', '2022-12-31')],
    'HE': [('2022-10-24', '2022-10-29'), ('2022-12-22', '2022-12-31')],
    'MV': [('2022-10-10', '2022-10-14'), ('2022-11-01', '2022-11-02'), ('2022-12-22', '2022-12-31')],
    'NI': [('2022-10-17', '2022-10-28'), ('2022-12-23', '2022-12-31')],
    'NW': [('2022-10-04', '2022-10-15'), ('2022-12-23', '2022-12-31')],
    'RP': [('2022-10-17', '2022-10-31'), ('2022-12-23', '2022-12-31')],
    'SL': [('2022-10-24', '2022-11-04'), ('2022-12-22', '2022-12-31')],
    'SN': [('2022-10-17', '2022-10-29'), ('2022-12-22', '2022-12-31')],
    'ST': [('2022-10-24', '2022-11-04'), ('2022-12-21', '2022-12-31')],
    'SH': [('2022-10-10', '2022-10-21'), ('2022-12-23', '2022-12-31')],
    'TH': [('2022-10-17', '2022-10-29'), ('2022-12-22', '2022-12-31')],
}
DE_hol_2023_1_dict = {
    'BW': [('2023-01-01', '2023-01-07'), ('2023-04-06', '2023-04-06'), ('2023-04-11', '2023-04-15'), ('2023-05-30', '2023-06-09')],
    'BY': [('2023-01-01', '2023-01-07'), ('2023-02-20', '2023-02-24'), ('2023-04-03', '2023-04-15'), ('2023-05-30', '2023-06-09')],
    'BE': [('2023-01-01', '2023-01-02'), ('2023-01-30', '2023-02-04'), ('2023-04-03', '2023-04-14'), ('2023-05-19', '2023-05-19'), ('2023-05-30', '2023-05-30')],
    'BB': [('2023-01-01', '2023-01-03'), ('2023-01-30', '2023-02-03'), ('2023-04-03', '2023-04-14')],
    'HB': [('2023-01-01', '2023-01-06'), ('2023-01-30', '2023-01-31'), ('2023-03-27', '2023-04-11'), ('2023-05-19', '2023-05-19'), ('2023-05-30', '2023-05-30')],
    'HH': [('2023-01-01', '2023-01-06'), ('2023-01-27', '2023-01-27'), ('2023-03-06', '2023-03-17'), ('2023-05-15', '2023-05-19')],
    'HE': [('2023-01-01', '2023-01-07'), ('2023-04-03', '2023-04-22')],
    'MV': [('2023-01-01', '2023-01-02'), ('2023-02-06', '2023-02-18'), ('2023-04-03', '2023-04-12'), ('2023-05-19', '2023-05-19'), ('2023-05-26', '2023-05-30')],
    'NI': [('2023-01-01', '2023-01-06'), ('2023-01-30', '2023-01-31'), ('2023-03-27', '2023-04-11'), ('2023-05-19', '2023-05-19'), ('2023-05-30', '2023-05-30')],
    'NW': [('2023-01-01', '2023-01-06'), ('2023-04-03', '2023-04-15'), ('2023-05-30', '2023-05-30')],
    'RP': [('2023-01-01', '2023-01-02'), ('2023-04-03', '2023-04-06'), ('2023-05-30', '2023-06-07')],
    'SL': [('2023-01-01', '2023-01-04'), ('2023-02-20', '2023-02-24'), ('2023-04-03', '2023-04-12'), ('2023-05-30', '2023-06-02')],
    'SN': [('2023-01-01', '2023-01-02'), ('2023-02-13', '2023-02-24'), ('2023-04-07', '2023-04-15'), ('2023-05-19', '2023-05-19')],
    'ST': [('2023-01-01', '2023-01-05'), ('2023-02-06', '2023-02-11'), ('2023-04-03', '2023-04-08'), ('2023-05-15', '2023-05-19')],
    'SH': [('2023-01-01', '2023-01-07'), ('2023-04-06', '2023-04-22'), ('2023-05-19', '2023-05-20')],
    'TH': [('2023-01-01', '2023-01-03'), ('2023-02-13', '2023-02-17'), ('2023-04-03', '2023-04-15'), ('2023-05-19', '2023-05-19')],
}
DE_hol_2023_2_dict = {
    'BW': [('2023-10-31', '2023-11-03'), ('2023-12-23', '2023-12-31')],
    'BY': [('2023-10-30', '2023-11-03'), ('2023-12-23', '2023-12-31')],
    'BE': [('2023-10-02', '2023-10-02'), ('2023-10-23', '2023-11-04'), ('2023-12-23', '2023-12-31')],
    'BB': [('2023-10-23', '2023-11-04'), ('2023-12-23', '2023-12-31')],
    'HB': [('2023-10-02', '2023-10-02'), ('2023-10-16', '2023-10-30'), ('2023-12-23', '2023-12-31')],
    'HH': [('2023-10-02', '2023-10-02'), ('2023-10-16', '2023-10-27'), ('2023-12-22', '2023-12-31')],
    'HE': [('2023-10-23', '2023-10-28'), ('2023-12-27', '2023-12-31')],
    'MV': [('2023-10-09', '2023-10-14'), ('2023-10-30', '2023-11-01'), ('2023-12-21', '2023-12-31')],
    'NI': [('2023-10-02', '2023-10-02'), ('2023-10-16', '2023-10-30'), ('2023-12-27', '2023-12-31')],
    'NW': [('2023-10-02', '2023-10-14'), ('2023-12-21', '2023-12-31')],
    'RP': [('2023-10-16', '2023-10-27'), ('2023-12-27', '2023-12-31')],
    'SL': [('2023-10-23', '2023-11-03'), ('2023-12-21', '2023-12-31')],
    'SN': [('2023-10-02', '2023-10-14'), ('2023-10-30', '2023-10-30'), ('2023-12-23', '2023-12-31')],
    'ST': [('2023-10-02', '2023-10-02'), ('2023-10-16', '2023-10-30'), ('2023-12-21', '2023-12-31')],
    'SH': [('2023-10-16', '2023-10-27'), ('2023-12-27', '2023-12-31')],
    'TH': [('2023-10-02', '2023-10-14'), ('2023-12-22', '2023-12-31')],
}
DE_hol_2024_1_dict = {
    'BW': [('2024-01-01', '2024-01-05'), ('2024-03-23', '2024-04-05'), ('2024-05-21', '2024-05-31')],
    'BY': [('2024-01-01', '2024-01-05'), ('2024-02-12', '2024-02-16'), ('2024-03-25', '2024-04-06'), ('2024-05-21', '2024-06-01')],
    'BE': [('2024-01-01', '2024-01-05'), ('2024-02-05', '2024-02-10'), ('2024-03-25', '2024-04-05'), ('2024-05-10', '2024-05-10')],
    'BB': [('2024-01-01', '2024-01-05'), ('2024-02-05', '2024-02-09'), ('2024-03-25', '2024-04-05')],
    'HB': [('2024-01-01', '2024-01-05'), ('2024-02-01', '2024-02-02'), ('2024-03-18', '2024-03-28'), ('2024-05-10', '2024-05-10')],
    'HH': [('2024-01-01', '2024-01-05'), ('2024-02-02', '2024-02-02'), ('2024-03-18', '2024-03-28'), ('2024-05-10', '2024-05-10'), ('2024-05-21', '2024-05-24')],
    'HE': [('2024-01-01', '2024-01-13'), ('2024-03-25', '2024-04-13')],
    'MV': [('2024-01-01', '2024-01-03'), ('2024-02-05', '2024-02-16'), ('2024-03-25', '2024-04-03'), ('2024-05-10', '2024-05-10'), ('2024-05-17', '2024-05-21')],
    'NI': [('2024-01-01', '2024-01-05'), ('2024-02-01', '2024-02-02'), ('2024-03-18', '2024-03-28'), ('2024-05-10', '2024-05-10'), ('2024-05-21', '2024-05-21')],
    'NW': [('2024-01-01', '2024-01-05'), ('2024-03-25', '2024-04-06'), ('2024-05-21', '2024-05-21')],
    'RP': [('2024-01-01', '2024-01-05'), ('2024-03-25', '2024-04-02'), ('2024-05-21', '2024-05-29')],
    'SL': [('2024-01-01', '2024-01-02'), ('2024-02-12', '2024-02-16'), ('2024-03-25', '2024-04-05'), ('2024-05-21', '2024-05-24')],
    'SN': [('2024-01-01', '2024-01-02'), ('2024-02-12', '2024-02-23'), ('2024-03-28', '2024-04-05'), ('2024-05-10', '2024-05-10'), ('2024-05-18', '2024-05-21')],
    'ST': [('2024-01-01', '2024-01-03'), ('2024-02-05', '2024-02-10'), ('2024-03-25', '2024-03-30'), ('2024-05-21', '2024-05-24')],
    'SH': [('2024-01-01', '2024-01-06'), ('2024-04-02', '2024-04-19'), ('2024-05-10', '2024-05-11')],
    'TH': [('2024-01-01', '2024-01-05'), ('2024-02-12', '2024-02-16'), ('2024-03-25', '2024-04-06'), ('2024-05-10', '2024-05-10')],
}
DE_hol_2024_2_dict = {
    'BW': [('2024-10-28', '2024-10-31'), ('2024-12-23', '2024-12-31')],
    'BY': [('2024-10-28', '2024-10-31'), ('2024-12-23', '2024-12-31')],
    'BE': [('2024-10-04', '2024-10-04'), ('2024-10-21', '2024-11-02'), ('2024-12-23', '2024-12-31')],
    'BB': [('2024-10-21', '2024-11-02'), ('2024-12-23', '2024-12-31')],
    'HB': [('2024-10-04', '2024-10-19'), ('2024-11-01', '2024-11-01'), ('2024-12-23', '2024-12-31')],
    'HH': [('2024-10-04', '2024-10-04'), ('2024-10-21', '2024-11-01'), ('2024-12-20', '2024-12-31')],
    'HE': [('2024-10-14', '2024-10-25'), ('2024-12-23', '2024-12-31')],
    'MV': [('2024-10-04', '2024-10-04'), ('2024-10-21', '2024-10-26'), ('2024-11-01', '2024-11-01'), ('2024-12-23', '2024-12-31')],
    'NI': [('2024-10-04', '2024-10-19'), ('2024-11-01', '2024-11-01'), ('2024-12-23', '2024-12-31')],
    'NW': [('2024-10-14', '2024-10-26'), ('2024-12-23', '2024-12-31')],
    'RP': [('2024-10-14', '2024-10-25'), ('2024-12-23', '2024-12-31')],
    'SL': [('2024-10-14', '2024-10-25'), ('2024-12-23', '2024-12-31')],
    'SN': [('2024-10-07', '2024-10-19'), ('2024-12-23', '2024-12-31')],
    'ST': [('2024-09-30', '2024-10-12'), ('2024-11-01', '2024-11-01'), ('2024-12-23', '2024-12-31')],
    'SH': [('2024-10-21', '2024-11-01'), ('2024-12-19', '2024-12-31')],
    'TH': [('2024-09-30', '2024-10-12'), ('2024-12-23', '2024-12-31')],
}
DE_hol_2025_1_dict = {
    'BW': [('2025-01-01', '2025-01-04'), ('2025-04-14', '2025-04-26'), ('2025-06-10', '2025-06-20'),
           ('2025-07-31', '2025-09-13')],
    'BY': [('2025-01-01', '2025-01-03'), ('2025-03-03', '2025-03-07'), ('2025-04-14', '2025-04-25'), 
           ('2025-06-10', '2025-06-20'), ('2025-08-01', '2025-08-01'), ('2025-08-04', '2025-09-15')],
    'BE': [('2025-01-01', '2025-01-01'), ('2025-02-03', '2025-02-08'), ('2025-04-14', '2025-04-25'), 
           ('2025-05-02', '2025-05-02'), ('2025-05-30', '2025-05-30'), ('2025-06-10', '2025-06-10'),
           ('2025-07-24', '2025-09-06')],
    'BB': [('2025-01-01', '2025-01-01'), ('2025-02-03', '2025-02-08'), ('2025-04-14', '2025-04-25'), 
           ('2025-06-10', '2025-06-10'), ('2025-07-24', '2025-09-06')],
    'HB': [('2025-01-01', '2025-01-04'), ('2025-02-03', '2025-02-04'), ('2025-04-07', '2025-04-19'), 
           ('2025-04-30', '2025-04-30'), ('2025-05-02', '2025-05-02'), ('2025-05-30', '2025-05-30'), 
           ('2025-06-10', '2025-06-10'), ('2025-07-03', '2025-08-13')],
    'HH': [('2025-01-01', '2025-01-03'), ('2025-01-31', '2025-01-31'), 
           ('2025-03-10', '2025-03-21'), ('2025-05-02', '2025-05-02'), 
           ('2025-05-26', '2025-05-30'), ('2025-07-24', '2025-09-03')],
    'HE': [('2025-01-01', '2025-01-10'), ('2025-04-07', '2025-04-21'), ('2025-07-07', '2025-08-15')],
    'MV': [('2025-01-01', '2025-01-06'), ('2025-02-03', '2025-02-14'), ('2025-04-14', '2025-04-23'), 
           ('2025-05-30', '2025-05-30'), ('2025-06-06', '2025-06-10'), ('2025-07-28', '2025-09-06')],
    'NI': [('2025-01-01', '2025-01-04'), ('2025-02-03', '2025-02-04'), ('2025-04-07', '2025-04-19'), 
           ('2025-04-30', '2025-04-30'), ('2025-05-02', '2025-05-02'), ('2025-05-30', '2025-05-30'), 
           ('2025-06-10', '2025-06-10'), ('2025-07-03', '2025-08-13')],
    'NW': [('2025-01-01', '2025-01-06'), ('2025-04-14', '2025-04-26'), ('2025-06-10', '2025-06-10'),
           ('2025-07-14', '2025-08-26') ],
    'RP': [('2025-01-01', '2025-01-08'), ('2025-04-14', '2025-04-25'), ('2025-07-07', '2025-08-15')],
    'SL': [('2025-01-01', '2025-01-03'), ('2025-02-24', '2025-03-04'), ('2025-04-14', '2025-04-25'),
           ('2025-07-07', '2025-08-14')],
    'SN': [('2025-01-01', '2025-01-03'), ('2025-02-17', '2025-03-01'), ('2025-04-18', '2025-04-25'), 
           ('2025-05-30', '2025-05-30'), ('2025-06-28', '2025-08-08')],
    'ST': [('2025-01-01', '2025-01-04'), ('2025-01-27', '2025-01-31'), ('2025-04-07', '2025-04-19'), 
           ('2025-05-30', '2025-05-30'), ('2025-06-28', '2025-08-08')],
    'SH': [('2025-01-01', '2025-01-07'), ('2025-04-11', '2025-04-25'), ('2025-05-30', '2025-05-30'), 
           ('2025-07-28', '2025-09-06')],
    'TH': [('2025-01-01', '2025-01-03'), ('2025-02-03', '2025-02-08'), ('2025-04-07', '2025-04-19'), 
           ('2025-05-30', '2025-05-30'), ('2025-06-28', '2025-08-08')],
}
DE_hol_2025_2_dict = {
    'BW': [('2025-10-27', '2025-10-31'), ('2025-12-22', '2025-12-31')],
    'BY': [('2025-11-03', '2025-11-07'), ('2025-12-22', '2025-12-31')],
    'BE': [('2025-10-20', '2025-11-01'), ('2025-12-22', '2025-12-31')],
    'BB': [('2025-10-20', '2025-11-01'), ('2025-12-22', '2025-12-31')],
    'HB': [('2025-10-13', '2025-10-25'), ('2025-12-22', '2025-12-31')],
    'HH': [('2025-10-20', '2025-10-31'), ('2025-12-17', '2025-12-31')],
    'HE': [('2025-10-06', '2025-10-18'), ('2025-12-22', '2025-12-31')],
    'MV': [('2025-10-02', '2025-10-02'), ('2025-10-20', '2025-10-25'), ('2025-11-03', '2025-11-03'), ('2025-12-22', '2025-12-31')],
    'NI': [('2025-10-13', '2025-10-25'), ('2025-12-22', '2025-12-31')],
    'NW': [('2025-10-13', '2025-10-25'), ('2025-12-22', '2025-12-31')],
    'RP': [('2025-10-13', '2025-10-24'), ('2025-12-22', '2025-12-31')],
    'SL': [('2025-10-13', '2025-10-24'), ('2025-12-22', '2025-12-31')],
    'SN': [('2025-10-06', '2025-10-18'), ('2025-12-22', '2025-12-31')],
    'ST': [('2025-10-13', '2025-10-25'), ('2025-12-22', '2025-12-31')],
    'SH': [('2025-10-20', '2025-10-30'), ('2025-12-19', '2025-12-31')],
    'TH': [('2025-10-06', '2025-10-18'), ('2025-12-22', '2025-12-31')],
}
DE_hol_2026_1_dict = {
    'BW': [('2026-01-01', '2026-01-05'), ('2026-03-30', '2026-04-11'), ('2026-05-26', '2026-06-05'), ('2026-07-30', '2026-09-12')],
    'BY': [('2026-01-01', '2026-01-05'), ('2026-02-16', '2026-02-20'), ('2026-03-30', '2026-04-10'), ('2026-05-26', '2026-06-05'), ('2026-08-03', '2026-09-14')],
    'BE': [('2026-01-01', '2026-01-02'), ('2026-02-02', '2026-02-07'), ('2026-03-30', '2026-04-10'), ('2026-05-15', '2026-05-15'), ('2026-06-26', '2026-06-26'), ('2026-07-09', '2026-08-22')],
    'BB': [('2026-01-01', '2026-01-02'), ('2026-02-02', '2026-02-07'), ('2026-03-30', '2026-04-10'), ('2026-05-26', '2026-05-26'), ('2026-07-09', '2026-08-22')],
    'HB': [('2026-01-01', '2026-01-05'), ('2026-02-02', '2026-02-03'), ('2026-03-23', '2026-04-07'), ('2026-05-15', '2026-05-15'), ('2026-05-26', '2026-05-26'), ('2026-07-02', '2026-08-12')],
    'HH': [('2026-01-01', '2026-01-02'), ('2026-01-30', '2026-01-30'), ('2026-03-02', '2026-03-13'), ('2026-05-11', '2026-05-15'), ('2026-07-09', '2026-08-19')],
    'HE': [('2026-01-01', '2026-01-10'), ('2026-03-30', '2026-04-10'), 
           ('2026-06-29', '2026-08-07')],
    'MV': [('2026-01-01', '2026-01-05'), ('2026-02-09', '2026-02-20'), ('2026-03-30', '2026-04-08'), ('2026-05-15', '2026-05-15'), ('2026-05-22', '2026-05-26'), ('2026-07-13', '2026-08-22')],
    'NI': [('2026-01-01', '2026-01-05'), ('2026-02-02', '2026-02-03'), ('2026-03-23', '2026-04-07'), ('2026-05-15', '2026-05-15'), ('2026-05-26', '2026-05-26'), ('2026-07-02', '2026-08-12')],
    'NW': [('2026-01-01', '2026-01-06'), ('2026-03-30', '2026-04-11'), ('2026-05-26', '2026-05-26'), ('2026-07-20', '2026-09-01')],
    'RP': [('2026-01-01', '2026-01-07'), ('2026-03-30', '2026-04-10'), ('2026-06-29', '2026-08-07')],
    'SL': [('2026-01-01', '2026-01-02'), ('2026-02-16', '2026-02-20'), ('2026-04-07', '2026-04-17'), ('2026-06-26', '2026-08-07')],
    'SN': [('2026-01-01', '2026-01-02'), ('2026-02-09', '2026-02-21'), ('2026-04-03', '2026-04-10'), ('2026-05-15', '2026-05-15'), ('2026-07-04', '2026-08-14')],
    'ST': [('2026-01-01', '2026-01-05'), ('2026-01-31', '2026-02-06'), ('2026-03-30', '2026-04-04'), ('2026-05-26', '2026-05-29'), ('2026-07-04', '2026-08-14')],
    'SH': [('2026-01-01', '2026-01-06'), ('2026-03-26', '2026-04-10'), ('2026-05-15', '2026-05-15'), ('2026-07-04', '2026-08-15')],
    'TH': [('2026-01-01', '2026-01-03'), ('2026-02-16', '2026-02-21'), ('2026-04-07', '2026-04-17'), ('2026-05-15', '2026-05-15'), ('2026-07-04', '2026-08-14')],
}


In [ ]:
DE_hol_2026_2_dict = {
    'BW': [('2026-10-26', '2026-10-31'), ('2026-12-23', '2026-12-31')],
    'BY': [('2026-11-02', '2026-11-06'), ('2026-12-24', '2026-12-31')],
    'BE': [('2026-10-19', '2026-10-31'), ('2026-12-23', '2026-12-31')],
    'BB': [('2026-10-19', '2026-10-30'), ('2026-12-23', '2026-12-31')],
    'HB': [('2026-10-12', '2026-10-24'), ('2026-12-23', '2026-12-31')],
    'HH': [('2026-10-19', '2026-10-30'), ('2026-12-21', '2026-12-31')],
    'HE': [('2026-10-05', '2026-10-17'), ('2026-12-23', '2026-12-31')],
    'MV': [('2026-10-19', '2026-10-24'), ('2026-11-26', '2026-11-27'), ('2026-12-19', '2026-12-31')],
    'NI': [('2026-10-12', '2026-10-24'), ('2026-12-23', '2026-12-31')],
    'NW': [('2026-10-17', '2026-10-31'), ('2026-12-23', '2026-12-31')],
    'RP': [('2026-10-05', '2026-10-16'), ('2026-12-23', '2026-12-31')],
    'SL': [('2026-10-05', '2026-10-16'), ('2026-12-21', '2026-12-31')],
    'SN': [('2026-10-12', '2026-10-24'), ('2026-12-23', '2026-12-31')],
    'ST': [('2026-10-19', '2026-10-30'), ('2026-12-21', '2026-12-31')],
    'SH': [('2026-10-12', '2026-10-24'), ('2026-12-21', '2026-12-31')],
    'TH': [('2026-10-12', '2026-10-24'), ('2026-12-23', '2026-12-31')],
}
DE_hol_2027_1_dict = {
    'BW': [('2027-01-01', '2027-01-09'), ('2027-03-25', '2027-03-25'), ('2027-03-30', '2027-04-03'), ('2027-05-18', '2027-05-29'), ('2027-07-29', '2027-09-11')],
    'BY': [('2027-01-01', '2027-01-08'), ('2027-02-08', '2027-02-12'), ('2027-03-22', '2027-04-02'), ('2027-05-18', '2027-05-28'), ('2027-08-02', '2027-09-13')],
    'BE': [('2027-01-01', '2027-01-02'), ('2027-02-01', '2027-02-06'), ('2027-03-22', '2027-04-02'), ('2027-05-07', '2027-05-07'), ('2027-05-18', '2027-05-19'), ('2027-07-01', '2027-08-14')],
    'BB': [('2027-01-01', '2027-01-02'), ('2027-02-01', '2027-02-06'), ('2027-03-22', '2027-04-03'), ('2027-05-18', '2027-05-18'), ('2027-07-01', '2027-08-14')],
    'HB': [('2027-01-01', '2027-01-09'), ('2027-02-01', '2027-02-02'), ('2027-03-22', '2027-04-03'), ('2027-05-07', '2027-05-07'), ('2027-05-18', '2027-05-18'), ('2027-07-08', '2027-08-18')],
    'HH': [('2027-01-01', '2027-01-01'), ('2027-01-29', '2027-01-29'), ('2027-03-01', '2027-03-12'), ('2027-05-07', '2027-05-14'), ('2027-07-01', '2027-08-11')],
    'HE': [('2027-01-01', '2027-01-12'), ('2027-03-22', '2027-04-02'), ('2027-06-28', '2027-08-06')],
    'MV': [('2027-01-01', '2027-01-02'), ('2027-02-08', '2027-02-19'), ('2027-03-22', '2027-03-31'), ('2027-05-07', '2027-05-07'), ('2027-05-14', '2027-05-18'), ('2027-07-05', '2027-08-14')],
    'NI': [('2027-01-01', '2027-01-09'), ('2027-02-01', '2027-02-02'), ('2027-03-22', '2027-04-03'), ('2027-05-07', '2027-05-07'), ('2027-05-18', '2027-05-18'), ('2027-07-08', '2027-08-18')],
    'NW': [('2027-01-01', '2027-01-06'), ('2027-03-22', '2027-04-03'), ('2027-05-18', '2027-05-18'), ('2027-07-19', '2027-08-31')],
    'RP': [('2027-01-01', '2027-01-08'), ('2027-03-22', '2027-04-02'), ('2027-06-28', '2027-08-06')],
    'SL': [('2027-01-01', '2027-01-05'), ('2027-02-08', '2027-02-12'), ('2027-03-30', '2027-04-09'), ('2027-06-28', '2027-08-06')],
    'SN': [('2027-01-01', '2027-01-02'), ('2027-02-08', '2027-02-19'), ('2027-03-26', '2027-04-02'), ('2027-05-07', '2027-05-07'), ('2027-05-15', '2027-05-18'), ('2027-07-10', '2027-08-20')],
    'ST': [('2027-01-01', '2027-01-02'), ('2027-02-01', '2027-02-06'), ('2027-03-22', '2027-03-27'), ('2027-05-15', '2027-05-22'), ('2027-07-10', '2027-08-20')],
    'SH': [('2027-01-01', '2027-01-06'), ('2027-03-30', '2027-04-10'), ('2027-05-07', '2027-05-07'), ('2027-07-03', '2027-08-14')],
    'TH': [('2027-01-01', '2027-01-02'), ('2027-02-01', '2027-02-06'), ('2027-03-22', '2027-04-03'), ('2027-05-07', '2027-05-07'), ('2027-07-10', '2027-08-20')],
}

**Baden-Württemberg** has its school holidays defined until the summer of **2029**, **Bavaria** even until the summer of **2030** :

In [ ]:
DE_hol_2027_2_dict = {
    'BW': [('2027-11-02', '2027-11-06'), ('2027-12-23', '2027-12-31')],
    'BY': [('2027-11-02', '2027-11-05'), ('2027-12-24', '2027-12-31')]   }
DE_hol_2028_1_dict = {
    'BW': [('2028-01-01', '2028-01-08'), ('2028-04-13', '2028-04-22'), 
           ('2028-06-06', '2028-06-17'), ('2028-07-27', '2028-09-09')],
    'BY': [('2028-01-01', '2028-01-07'), ('2028-02-28', '2028-03-03'), 
           ('2028-04-10', '2028-04-21'), ('2028-06-06', '2028-06-16'), 
           ('2028-07-31', '2028-09-11')]   }
DE_hol_2028_2_dict = {
    'BW': [('2028-10-30', '2028-11-03'), ('2028-12-23', '2028-12-31')],
    'BY': [('2028-10-30', '2028-11-03'), ('2028-12-23', '2028-12-31')]   }
DE_hol_2029_1_dict = {
    'BW': [('2029-01-01', '2029-01-05')],
    'BY': [('2029-01-01', '2029-01-05'), ('2029-02-12', '2029-02-16'), 
           ('2029-03-26', '2029-04-06'), ('2029-05-22', '2029-06-01'), 
           ('2029-07-30', '2029-09-10')]   }
DE_hol_2029_2_dict = {
    'BY': [('2029-10-29', '2029-11-02'), ('2029-12-24', '2029-12-31')]   }
DE_hol_2030_1_dict = {
    'BY': [('2030-01-01', '2030-01-04'), ('2030-03-04', '2030-03-08'),
           ('2030-04-15', '2030-04-26'), ('2030-06-11', '2030-06-21'), 
           ('2030-07-29', '2030-09-09')]   }

<h3> Update the DataFrame's School Holiday Columns </h3>

The function `update_school_DE_hol_df_fct` takes the DataFrame  `DE_hol_df` and, for every federal state, flags the days lying in one of the school holiday dictionaries' holiday intervals for that state as --- a holiday.

In [10]:
from pathlib import Path
import sys
sys.path.insert(0, str(Path.cwd().parent))
from src.data_modules.german_school_semesters import SEMESTER_REGISTRY

DE_hol_df = update_school_DE_hol_df_fct(
    2015, 2020,
    DE_hol_df,
    semester_registry=SEMESTER_REGISTRY,
    add_weekends=True,                   # set False if you don't want weekends
)

In [13]:
DE_hol_df.head(40)

,local_date,local_start,local_end,utc_start,utc_end,DE_BW_hol,DE_BW_school_free,DE_BY_hol,DE_BY_school_free,DE_BE_hol,...,DE_SL_hol,DE_SL_school_free,DE_SN_hol,DE_SN_school_free,DE_ST_hol,DE_ST_school_free,DE_SH_hol,DE_SH_school_free,DE_TH_hol,DE_TH_school_free
0,2015-01-01,2015-01-01 00:00:00+01:00,2015-01-02 00:00:00+01:00,2014-12-31 23:00:00+00:00,2015-01-01 23:00:00+00:00,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1
1,2015-01-02,2015-01-02 00:00:00+01:00,2015-01-03 00:00:00+01:00,2015-01-01 23:00:00+00:00,2015-01-02 23:00:00+00:00,0,1,0,1,0,...,0,1,0,1,0,1,0,1,0,1
2,2015-01-03,2015-01-03 00:00:00+01:00,2015-01-04 00:00:00+01:00,2015-01-02 23:00:00+00:00,2015-01-03 23:00:00+00:00,0,1,0,1,0,...,0,1,0,1,0,1,0,1,0,1
3,2015-01-04,2015-01-04 00:00:00+01:00,2015-01-05 00:00:00+01:00,2015-01-03 23:00:00+00:00,2015-01-04 23:00:00+00:00,0,1,0,1,0,...,0,1,0,1,0,1,0,1,0,1
4,2015-01-05,2015-01-05 00:00:00+01:00,2015-01-06 00:00:00+01:00,2015-01-04 23:00:00+00:00,2015-01-05 23:00:00+00:00,0,1,0,1,0,...,0,1,0,0,0,1,0,1,0,0
5,2015-01-06,2015-01-06 00:00:00+01:00,2015-01-07 00:00:00+01:00,2015-01-05 23:00:00+00:00,2015-01-06 23:00:00+00:00,1,1,1,1,0,...,0,1,0,0,1,1,0,1,0,0
6,2015-01-07,2015-01-07 00:00:00+01:00,2015-01-08 00:00:00+01:00,2015-01-06 23:00:00+00:00,2015-01-07 23:00:00+00:00,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
7,2015-01-08,2015-01-08 00:00:00+01:00,2015-01-09 00:00:00+01:00,2015-01-07 23:00:00+00:00,2015-01-08 23:00:00+00:00,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8,2015-01-09,2015-01-09 00:00:00+01:00,2015-01-10 00:00:00+01:00,2015-01-08 23:00:00+00:00,2015-01-09 23:00:00+00:00,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9,2015-01-10,2015-01-10 00:00:00+01:00,2015-01-11 00:00:00+01:00,2015-01-09 23:00:00+00:00,2015-01-10 23:00:00+00:00,0,1,0,1,0,...,0,1,0,1,0,1,0,1,0,1
